In [1]:
pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=4b265b8b8e48db982820884846efb96567d2e275f63547614111cca1d5128a27
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import numpy as np
#받은 데이터들을 행렬로써 이용할 것이기에 행렬 연산을 용이하게 하는 numpy사용

# 분류용 샘플 데이터 불러오기
#문자열 데이터를 가져오는 출처가 fetch_20newsgroups()인 것이다. 다른 데아터를 가져 올 수도 있다.
news = fetch_20newsgroups()
X, y, labels = news.data, news.target, news.target_names

# 학습/테스트 데이터셋 분할
#train_test_split은 학습 모듈과 테스트 데이터를 나누는 함수이다. 
#X, y 2개의 데이터가 주어지면 X_train, X_test, y_train, y_test의 4개의 결과값이 나타난다. 
#test_size는 test와 train의 비율을 나타낸다. 0.3일 경우 테스트데이터는 전체 데이터의 30%이다.
#random_state는 0일 경우 테스트 데이터가 고정되어 나타나고 1일 경우 테스트 데이터가 유동적으로 바뀌어 나타난다.
#stratify는 Label Set인 y가 25%의 0과 75%의 1로 이루어진 Binary Set일 때, stratify=y로 설정하면 나누어진 데이터셋들도 0과 1을 각각 25%, 75%로 유지한 채 분할된다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

# 데이터 전처리(벡터화)
#CountVectorizer()는 데이터 내 문자열들을 토큰화하여 나타내고 같은 토큰이 나타난 횟수도 나타내어 준다.
#TfidfTransformer()는 CountVectorizer()와 달리 단어의 중요도에 따라 가중치를 주기 위한 함수이다. 
#CountVectorizer()는 토큰의 출현횟수를 나타내는데에 반해 TfidfTransformer()는 데이터 내의 토큰의 출현 비율을 나타내기 때문에 중요도를 파악할 수 있다. 
vectorizer = CountVectorizer()
tfid = TfidfTransformer()

#fit은 최대,최소,평균,분산 등의 scaling에 필요한 정보를 파악하는 용도의 함수이며 transform은 실제로 그 파악한 정보를 통해 데이터셋의 값을 변환시키는 함수이다. 
#fit_transform은 이 둘을 동시에 실행시키는 함수이다. 그러므로 테스트 데이터에는 학습에 영향을 주지 않도록 fit을 사용하지 않고 학습 데이터에만 fit명령어를 포함시킨다.
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

X_train_tfid = tfid.fit_transform(X_train_vec)
X_test_tfid = tfid.transform(X_test_vec)

# 다중분류 나이브 베이브 + 그리드서치로 모델 학습
#np.linspace는 구간 시작점, 구간 끝점, 구간 내 숫자 개수를 받는다 0.01부터 1까지 100개의 숫자로 나타낸 것이다. 
#GridSearchCV는 하이퍼파라미터를 학습을 하고 측정을 하면서 최적화된 파라미터를 알려준다. 
#param_grid는 사용될 파라미터를 딕셔너리 형태로 만들어서 넣는다. scoring 은 파라미터에 대한 평가 방식에 해당한다.accuracy를 넣어 정확성을 평가한다.
#cv는 교차 검증에서 몇개로 분할되는지 지정한다. n_jobs는 매개 변수를 사용하여 사용할 코어 수를 지정할 수 있다. 사용하는 CPU 코어 개수에 비례해서 속도도 빨라진다.
nb = MultinomialNB()
param_grid = [{'alpha': np.linspace(0.01, 1, 100)}]
gs = GridSearchCV(estimator=nb, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
gs.fit(X_train_tfid, y_train)

# 그리드서치 학습 결과 출력
print('베스트 하이퍼 파라미터: {0}'.format(gs.best_params_))
print('베스트 하이퍼 파라미터 일 때 정확도: {0:.2f}'.format(gs.best_score_))

# 최적화 모델 추출
model = gs.best_estimator_

# 테스트세트 정확도 출력
score = model.score(X_test_tfid, y_test)
print('테스트세트에서의 정확도: {0:.2f}'.format(score))

# 테스트세트 예측 결과 샘플 출력
predicted_y = model.predict(X_test_tfid)
for i in range(10):
    print('실제 값: {0}, 예측 값: {1}'.format(labels[y_test[i]], labels[predicted_y[i]]))

베스트 하이퍼 파라미터: {'alpha': 0.01}
베스트 하이퍼 파라미터 일 때 정확도: 0.89
테스트세트에서의 정확도: 0.90
실제 값: talk.politics.misc, 예측 값: talk.politics.misc
실제 값: comp.sys.mac.hardware, 예측 값: comp.sys.mac.hardware
실제 값: sci.crypt, 예측 값: sci.crypt
실제 값: talk.politics.guns, 예측 값: talk.politics.guns
실제 값: alt.atheism, 예측 값: alt.atheism
실제 값: comp.windows.x, 예측 값: comp.windows.x
실제 값: sci.electronics, 예측 값: misc.forsale
실제 값: comp.sys.mac.hardware, 예측 값: comp.sys.mac.hardware
실제 값: sci.space, 예측 값: sci.space
실제 값: talk.politics.guns, 예측 값: talk.politics.guns
